# Teknik Feature Engineering


## Teknik Pemilihan Fitur (Feature Selection)

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_wine

In [2]:
data = load_wine()
X, y = data.data, data.target

In [3]:
df = pd.DataFrame(X, columns=data.feature_names)
df["target"] = y
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [4]:
# Pembagian data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Filter Method

In [5]:
# ------------------- Filter Methods -------------------
# Menggunakan SelectKBest
filter_selector = SelectKBest(score_func=chi2, k=2)  # Memilih 2 fitur terbaik
X_train_filter = filter_selector.fit_transform(X_train, y_train)
X_test_filter = filter_selector.transform(X_test)
 
print("Fitur yang dipilih dengan Filter Methods:", filter_selector.get_support(indices=True))

Fitur yang dipilih dengan Filter Methods: [ 9 12]


# Penjelasan SelectKBest dengan Chi-Squared (χ²)

## SelectKBest
**SelectKBest** adalah salah satu metode dari *filter methods* yang umum digunakan untuk memilih fitur terbaik berdasarkan skor statistik tertentu.

- **Parameter `score_func=chi2`**  
  Berarti metode ini menggunakan **Chi-squared (χ²)** sebagai fungsi skor.  
  Chi-squared adalah uji statistik yang digunakan untuk mengukur independensi antara dua variabel kategorikal.

- **Parameter `k=2`**  
  Artinya hanya akan dipilih **2 fitur terbaik** dari semua fitur yang ada di dataset.  
  Dengan kata lain, ini akan menyeleksi dua fitur dengan skor tertinggi berdasarkan hasil uji Chi-squared.

> Kedua parameter ini bisa Anda sesuaikan dengan studi kasus dan juga *trial and error*.

---

## fit_transform(X_train, y_train)
- **`fit(X_train, y_train)`**  
  Menghitung skor Chi-squared untuk setiap fitur pada data pelatihan (`X_train`) terhadap target (`y_train`).

- **`transform(X_train)`**  
  Menyalin data dari `X_train`, tetapi hanya menyimpan fitur yang dipilih oleh `filter_selector`.

Hasilnya adalah dataset pelatihan baru (`X_train_filter`) yang hanya mengandung **dua fitur terbaik** hasil perhitungan Chi-squared.

---

## transform(X_test)
Fungsi **`transform(X_test)`** digunakan untuk mentransformasikan dataset uji (`X_test`) dengan memilih fitur yang sama seperti pada dataset pelatihan.

---

## get_support(indices=True)
Fungsi **`get_support(indices=True)`** digunakan untuk mendapatkan **indeks fitur** yang dipilih.  
Dalam hal ini, fitur yang dipilih adalah **dua fitur dengan skor tertinggi** berdasarkan uji Chi-squared.

Output dari fungsi ini akan mencetak indeks fitur dari dataset awal sehingga kita tahu fitur mana yang tetap dipertahankan setelah proses seleksi.

---

## Hasil Akhir
- Dataset yang sudah difilter hanya berisi **dua fitur dengan skor independensi terbesar**.  
- Indeks fitur yang dipertahankan bisa digunakan untuk analisis lebih lanjut.  

---

📌 *Sampai di sini, Anda sudah memiliki dataset dengan dua fitur terbaik.  
Namun, tahan dulu rasa penasaran Anda, karena pada akhir materi ini kita akan membandingkan performa dari tiga metode feature selection.*


## Wrapper Method

In [6]:
# Menggunakan RFE (Recursive Feature Elimination)
model = LogisticRegression(solver='lbfgs', max_iter=5000)
rfe_selector = RFE(model, n_features_to_select=2)  # Memilih 2 fitur
X_train_rfe = rfe_selector.fit_transform(X_train, y_train)
X_test_rfe = rfe_selector.transform(X_test)
 
print("Fitur yang dipilih dengan Wrapper Methods:", rfe_selector.get_support(indices=True))

Fitur yang dipilih dengan Wrapper Methods: [0 6]


## Embedded Methods

In [7]:
from sklearn.ensemble import RandomForestClassifier
# ------------------- Embedded Methods -------------------
# Menggunakan Random Forest untuk mendapatkan fitur penting
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
 
# Mendapatkan fitur penting
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1]
 
# Menentukan ambang batas untuk fitur penting
threshold = 0.05  # Misalnya, ambang batas 5%
important_features_indices = [i for i in range(len(importances)) if importances[i] >= threshold]
 
# Memindahkan fitur penting ke variabel baru
X_important = X_train[:, important_features_indices]  # Hanya fitur penting dari data pelatihan
X_test_important = X_test[:, important_features_indices]  # Hanya fitur penting dari data pengujian
 
# Mencetak fitur yang dipilih
print("Fitur yang dipilih dengan Embedded Methods (di atas ambang batas):")
for i in important_features_indices:
    print(f"{data.feature_names[i]}: {importances[i]}")
 
# X_important sekarang berisi hanya fitur penting
print("\nDimensi data pelatihan dengan fitur penting:", X_important.shape)
print("Dimensi data pengujian dengan fitur penting:", X_test_important.shape)

Fitur yang dipilih dengan Embedded Methods (di atas ambang batas):
alcohol: 0.11239773542143086
flavanoids: 0.20229341635663622
color_intensity: 0.1712021830864957
hue: 0.07089132259413944
od280/od315_of_diluted_wines: 0.1115643167260497
proline: 0.13904586955351153

Dimensi data pelatihan dengan fitur penting: (142, 6)
Dimensi data pengujian dengan fitur penting: (36, 6)


In [8]:
# Evaluasi dengan fitur terpilih dari masing-masing metode
def evaluate_model(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    return accuracy

In [9]:
# Model Logistic Regression untuk Filter Methods
logistic_model_filter = LogisticRegression(max_iter=200)
accuracy_filter = evaluate_model(X_train_filter, X_test_filter, y_train, y_test, logistic_model_filter)
 
# Model Logistic Regression untuk Wrapper Methods
logistic_model_rfe = LogisticRegression(max_iter=200)
accuracy_rfe = evaluate_model(X_train_rfe, X_test_rfe, y_train, y_test, logistic_model_rfe)
 
# Model Random Forest untuk Embedded Methods
accuracy_rf = evaluate_model(X_important, X_test_important, y_train, y_test, rf_model)

In [10]:
print(f"\nAkurasi Model dengan Filter Methods: {accuracy_filter:.2f}")
print(f"Akurasi Model dengan Wrapper Methods: {accuracy_rfe:.2f}")
print(f"Akurasi Model dengan Embedded Methods: {accuracy_rf:.2f}")


Akurasi Model dengan Filter Methods: 0.89
Akurasi Model dengan Wrapper Methods: 0.94
Akurasi Model dengan Embedded Methods: 1.00
